## OPT Tutorial

### NOTE: this code cannot be run in the .ipynb, but you can copy and paste to your terminal for use

Our paper, "Evidence of off-target probe binding in the 10x Genomics Xenium v1 Human Breast Gene Expression Panel compromises accuracy of spatial transcriptomic profiling," discusses off-target binding in the Xenium v1 Human Breast Gene Expression Panel. Here, we will run through the OPT pipeline of how we obtained our results while also sharing how to effectively use OPT on your own probe sequences.

First, head to the [README.md](https://github.com/JEFworks-Lab/off-target-probe-tracker) and install the OPT pipeline with the necessary dependencies.

We have provided the 3 reference annotations: gencode, refseq, and chess, that were used in our paper. They are located in the "data" folder within the "off-target-probe-tracker" directory. You will need to unzip them to use them, which can be done as followed:

In [ ]:
# for gencode
cd data/gencode
gunzip *.gz

# for refseq
cd data/refseq
gunzip *.gz

# for chess
cd data/chess
gunzip *.gz

You will also need to unzip the xenium breast fasta file:

In [ ]:
cd data/probes
gunzip *.gz

If you have your own probe sequences you want to check, for consistency and so you can copy and paste this code best we recommend adding your probes in the "data/probes" folder. To note, the current expected header format for the probe sequence is:


`>gene_id|gene_name|accession`

### Run all three modules in one (`all` module)

As discussed in the [README.md](https://github.com/JEFworks-Lab/off-target-probe-tracker), there are 3 main modules in OPT: `flip`, `track`, and `stat`.

There is now an option to run them all at once with one command `all`. To better understand what each module does, please see below. However, it may be easier to simply run the `all` flag to acheive all desired output.

The `all` module allows you to run all three original modules in one. Your probes will be put in the forward orientation, aligned to your reference annotation of choice, and output statistics about this alignment. To run the xenium probes through the `all` module, you can run the following command:

In [ ]:
opt -o xenium_all/ -p 10 all -q data/probes/xenium_human_breast_gene_expression_panel_probe_sequences.fasta -a data/gencode/gencode.v47.basic.annotation.fmted.gff -t data/gencode/gencode.v47.basic.annotation.fmted.fa

The output shown in your terminal will look something like below, and all output of the opt pipeline will be in your -o folder 

In [ ]:
(opt) caleb:~/Desktop/off-target-probe-checker$ opt -o xenium_all/ -p 10 all -q data/probes/xenium_human_breast_gene_expression_panel_probe_sequences.fasta -a data/gencode/gencode.v47.basic.annotation.fmted.gff -t data/gencode/gencode.v47.basic.annotation.fmted.fa -s data/gene_synonyms.csv
2025-05-05 10:56:05.957232 PROGRESS ### ALL ###
2025-05-05 10:56:05.957372 PROGRESS aligning input probes to source transcripts
2025-05-05 10:56:05.957383 PROGRESS aligner: nucmer
nucmer  --maxmatch -l 20 -c 0 -t 10 data/gencode/gencode.v47.basic.annotation.fmted.fa data/probes/xenium_human_breast_gene_expression_panel_probe_sequences.fasta --sam-long=xenium_all/flip.sam
2025-05-05 10:56:49.955666 PROGRESS loading source transcriptome infos
2025-05-05 10:56:53.270354 PROGRESS loading input probes infos
2025-05-05 10:56:53.289568 PROGRESS 0 target gene IDs missing from the source
2025-05-05 10:56:53.289594 PROGRESS 7 target gene names missing from the source
2025-05-05 10:56:53.290831 PROGRESS parsing alignment results
2025-05-05 10:56:53.380356 PROGRESS 30 / 4809 probes unmapped
2025-05-05 10:56:53.381366 PROGRESS 7 / 4809 probes not mapped to their origin
2025-05-05 10:56:53.381377 PROGRESS 2508 / 4809 probes to flip (i.e., reverse complement)
2025-05-05 10:56:53.431334 PROGRESS wrote forward oriented probes to a file
2025-05-05 10:56:53.437136 PROGRESS aligning query probes to target transcripts
2025-05-05 10:56:53.437152 PROGRESS using fwd_oriented.fa probes
2025-05-05 10:56:53.437165 PROGRESS aligner: nucmer
nucmer -f --maxmatch -l 20 -c 0 -t 10 data/gencode/gencode.v47.basic.annotation.fmted.fa xenium_all/fwd_oriented.fa --sam-long=xenium_all/track.sam
2025-05-05 10:57:37.283630 PROGRESS loading target transcriptome infos
2025-05-05 10:57:40.579993 PROGRESS detecting potential off-target probe activities
2025-05-05 10:57:40.767242 PROGRESS 34 / 4809 probes unmapped
2025-05-05 10:57:40.782598 PROGRESS 15 / 4775 mapped probes with no passing hit
2025-05-05 10:57:40.782613 PROGRESS finished
2025-05-05 10:57:40.946247 PROGRESS number of probes missing targets: 4
2025-05-05 10:57:40.946354 PROGRESS number of probes with off-target binding: 206
2025-05-05 10:57:40.946459 PROGRESS number of probe genes: 280
2025-05-05 10:57:40.951622 PROGRESS 280 / 280 probe genes with at least 1 probe binding
2025-05-05 10:57:40.951843 PROGRESS number of missed probe genes: 0
2025-05-05 10:57:40.951852 PROGRESS number of off-target probe genes: 46

The `all` module can take as input the -pl, -1, --exclude-pseudo, --pc-only, -s, and -i flags. Please see the README or below for more details on these flags.

### Run all three modules separately

The `flip` module will allow all your probes to be on the same same strand orientation as your intended target gene. We highly recommend running all probes through this module to confirm all probes are set for downstream alignment. To run the xenium probes through the `flip` module, you can run the following command:

In [ ]:
opt -o xenium/ -p 10 flip -q data/probes/xenium_human_breast_gene_expression_panel_probe_sequences.fasta -a data/gencode/gencode.v47.basic.annotation.fmted.gff -t data/gencode/gencode.v47.basic.annotation.fmted.fa

Here, we are creating a new output directory named "xenium", allowing 10 cores to be used with the `-p` flag, and have input the needed files of the probe sequences `-q`, gff of reference annotation `-a`, and fa of reference annotation `-t`, in which we use gencode for this example.

This will output files in the "xenium" directory, one of which will be the `fwd_oriented.fa` that contains all probes in the correct orientation for OPT to proceed. You should see something like this in your terminal:

In [ ]:
(opt) caleb:~/Desktop/off-target-probe-tracker$ opt -o xenium/ -p 10 flip -q data/probes/xenium_human_breast_gene_expression_panel_probe_sequences.fasta -a data/gencode/gencode.v47.basic.annotation.fmted.gff -t data/gencode/gencode.v47.basic.annotation.fmted.fa
2025-04-02 15:07:32.709651 PROGRESS ### FLIP ###
2025-04-02 15:07:32.709843 PROGRESS aligning input probes to source transcripts
2025-04-02 15:07:32.709865 PROGRESS aligner: nucmer
nucmer  --maxmatch -l 20 -c 0 -t 10 data/gencode/gencode.v47.basic.annotation.fmted.fa data/probes/xenium_human_breast_gene_expression_panel_probe_sequences.fasta --sam-long=xenium/flip.sam
2025-04-02 15:08:25.704186 PROGRESS loading source transcriptome infos
2025-04-02 15:08:25.704405 PROGRESS building t2g mappings
2025-04-02 15:09:43.292936 PROGRESS loaded 158338 transcripts
2025-04-02 15:09:43.561811 PROGRESS loading input probes infos
2025-04-02 15:09:43.715986 PROGRESS 0 target gene IDs missing from the source
2025-04-02 15:09:43.716029 PROGRESS 7 target gene names missing from the source
2025-04-02 15:09:43.717948 PROGRESS parsing alignment results
2025-04-02 15:09:43.819012 PROGRESS 30 / 4809 probes unmapped
2025-04-02 15:09:43.820483 PROGRESS 7 / 4809 probes not mapped to their origin
2025-04-02 15:09:43.820496 PROGRESS 2508 / 4809 probes to flip (i.e., reverse complement)
2025-04-02 15:09:43.890430 PROGRESS wrote forward oriented probes to a file

Next, we will use the `track` module to align all the `fwd_oriented.fa` probes to the transcriptome as follows:

In [ ]:
opt -o xenium/ -p 10 track -q xenium/fwd_oriented.fa -a data/gencode/gencode.v47.basic.annotation.fmted.gff -t data/gencode/gencode.v47.basic.annotation.fmted.fa

Here, we are using our output directory named "xenium", allowing 10 cores to be used with the `-p` flag, and have input the needed files of the forward oriented probe sequences `-q`, gff of reference annotation `-a`, and fa of reference annotation `-t`, in which we use gencode for this example. The output on terminal should look something like this:

In [ ]:
(opt) caleb:~/Desktop/off-target-probe-tracker$ opt -o xenium/ -p 10 track -q xenium/fwd_oriented.fa -a data/gencode/gencode.v47.basic.annotation.fmted.gff -t data/gencode/gencode.v47.basic.annotation.fmted.fa
2025-04-02 15:14:12.364590 PROGRESS ### TRACK ###
2025-04-02 15:14:12.364837 PROGRESS aligning query probes to target transcripts
2025-04-02 15:14:12.364862 PROGRESS aligner: nucmer
nucmer -f --maxmatch -l 20 -c 0 -t 10 data/gencode/gencode.v47.basic.annotation.fmted.fa xenium/fwd_oriented.fa --sam-long=xenium/track.sam
2025-04-02 15:15:10.410369 PROGRESS loading target transcriptome infos
2025-04-02 15:15:10.410487 PROGRESS building t2g mappings
2025-04-02 15:16:37.142927 PROGRESS loaded 158338 transcripts
2025-04-02 15:16:37.425780 PROGRESS detecting potential off-target probe activities
2025-04-02 15:16:37.675444 PROGRESS 34 / 4809 probes unmapped
2025-04-02 15:16:37.690770 PROGRESS 15 / 4775 mapped probes with no passing hit
2025-04-02 15:16:37.690792 PROGRESS finished

This will output tsv file, `probe2targets.tsv`, that contains the gene and transcript information to which each probe aligns. To better understand this file and extract understable statistics, we run the `stat` module:

In [ ]:
opt -o xenium/ stat -i xenium/probe2targets.tsv -q xenium/fwd_oriented.fa -s data/gene_synonyms.csv

Here, we are using our output directory named "xenium" and have input the needed files: `probe2targets.tsv` `-i`, the forward oriented probes `-q`, and an optional parameter `-s` that allows you to provide gene synonyms that may have been counted as off-targets but simply differ in name. The terminal should again look something like this:

In [ ]:
(opt) caleb:~/Desktop/off-target-probe-tracker$ opt -o xenium/ stat -i xenium/probe2targets.tsv -q xenium/fwd_oriented.fa -s data/gene_synonyms.csv
2025-04-02 15:16:55.098035 PROGRESS ### STAT ###
2025-04-02 15:16:55.466949 PROGRESS number of probes missing targets: 4
2025-04-02 15:16:55.467188 PROGRESS number of probes with off-target binding: 206
2025-04-02 15:16:55.467432 PROGRESS number of probe genes: 280
2025-04-02 15:16:55.478718 PROGRESS 280 / 280 probe genes with at least 1 probe binding
2025-04-02 15:16:55.479132 PROGRESS number of missed probe genes: 0
2025-04-02 15:16:55.479147 PROGRESS number of off-target probe genes: 46

From the `stat` module we get a few important files that will be present in your output directory, which in our case is "xenium":
1. `collapsed_summary.tsv` - this will give us results similar to table 1 of the paper where we have the target gene, number of probes for that gene, which genes the probes aligned to, how many alignment hits, and how many probes were aligned to each gene. This is a great table to observe which genes are affected by off-target activity.
2. `probes2target.tsv` - this file is provided after the track module, but gives important information such as which probes aligned to which gene as well as the CIGAR string for these probes. It also shares what transcript type the probe is aligned to (ex. protein coding or lncRNA)
3. `stat_missed_genes.txt` and `stat_missed_probes.txt` - provide the genes and probes without alignments
4. `stat_off_target_genes.txt` and `stat_off_target_probes.txt` - provide the genes and probes with off-target activity
5. `stat_summary.tsv` - a table similar to `collapsed_summary.tsv` but all new-lines are one alignment of probes.
6. More to come!

Hopefully this tutorial shows you how to use and intepret OPT results. Another thing to note, the `-pl` flag can be used to change the number of errors allowed on the terminal ends of the probes during alignment - simply add this flag to the `track` module to obtain these results. Feel free to reach out or open an issue for questions/comments/concerns!